In [2]:
from __future__ import division

In [3]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp
import urllib
import os

In [51]:
from sqlalchemy import create_engine

In [5]:
pd.set_option('display.max_columns', None)
Gb = 1000000000

In [6]:
temp_csv = 'c:'+os.sep+'temp'+os.sep+'temp.log'

In [7]:
log_file = 'c:'+os.sep+'temp'+os.sep+'hk-ssg140.log-20150922'

In [55]:
disk_engine = create_engine('sqlite:///c:\\temp\\access_log.db')

In [9]:
#store = pd.HDFStore('access_log.h5')

In [10]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [11]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [12]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [13]:
#squid = sq.read_squid_log('access.log')

In [14]:
#squid.tail()

In [15]:
#safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [16]:
#access = squid[squid['URL'].str.contains(safe_websites) == False]

In [17]:
#sum(squid.bytes)/Gb

In [18]:
def clean_juniper_file(infile, outfile):
    with open(infile, 'r') as in_file:
        with open(outfile, 'w') as out_file:
            for line in in_file:
                if ('system-notification-00257(traffic)' in line):                                                
                    idx_1 = line.find('service=')
                    idx_2 = line.find(' proto=')
                    if (idx_1 and idx_2):
                        out_file.write(line[:idx_1]+line[idx_1:idx_2].replace(' ', '')+line[idx_2:])

In [19]:
clean_juniper_file(log_file, temp_csv)

In [20]:
def open_csv(filename, chunksize=20000):
    
    df = pd.DataFrame()
    
    for c in pd.read_table(filename, sep='\s{1,}|"', chunksize=chunksize, iterator=True, engine='python', index_col=False, header=None, parse_dates=[[0, 1, 2], [9,10]]):        
        df = pd.concat([df, c])        
        
    print(str(len(df)) + " rows processed.")
    
    return df

def juniper_log_cleansing(x):
    if str(x).find('=') > 0:
        return str(x)[str(x).find('=')+1:]
    elif str(x).find(':') > 0:
        return str(x)[:str(x).find(':')]    
    else:
        return x

def read_syslog_juniper(filename):
       
    df = open_csv(filename)
    
    df = df[df[7].str.contains('information') == False]
    
    # restructure the dataframe
    i = [0]
    i.extend(range(3, 4))
    i.extend([1])
    i.extend(range(9, 13))
    i.extend([14])
    i.extend([16])
    i.extend(range(17, 25))
    df = df.iloc[:, i]
    
    df.columns = ['time', 'device_name', 'traffic_start', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port', 'remarks']
    df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']] = df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']].applymap(juniper_log_cleansing)
    df[['policy_no', 'protocol', 'sent_size', 'received_size', 'src_port', 'dst_port']] = df[['policy_no', 'protocol', 'sent_size', 'received_size', 'src_port', 'dst_port']].convert_objects(convert_numeric = True)
    
    df = df.reset_index(drop=True)
    
    return df

def summary_groups(grouped):
    for name, group in grouped:
        print group
        

In [21]:
juniper = read_syslog_juniper(temp_csv)

1657993 rows processed.


In [23]:
juniper.head()

,time,device_name,traffic_start,duration,policy_no,service,protocol,src_zone,dst_zone,action,sent_size,received_size,src_address,dst_address,src_port,dst_port,remarks
0,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,203.80.96.10,57320,53,src-xlated
1,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,210.184.16.65,57320,53,src-xlated
2,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,210.0.128.250,57320,53,src-xlated
3,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,202.14.67.4,57320,53,src-xlated
4,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,208.109.255.8,57071,53,src-xlated


In [40]:
juniper['date'] = juniper.time.dt.date.astype(str)
juniper['total_size'] = juniper['sent_size'] + juniper['received_size']

In [41]:
juniper.head()

,time,device_name,traffic_start,duration,policy_no,service,protocol,src_zone,dst_zone,action,sent_size,received_size,src_address,dst_address,src_port,dst_port,remarks,date,total_size
0,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,203.80.96.10,57320,53,src-xlated,2015-09-21,0
1,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,210.184.16.65,57320,53,src-xlated,2015-09-21,0
2,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,210.0.128.250,57320,53,src-xlated,2015-09-21,0
3,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,202.14.67.4,57320,53,src-xlated,2015-09-21,0
4,2015-09-21 03:40:03,hk-SSG140,2015-09-21 03:52:07,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,208.109.255.8,57071,53,src-xlated,2015-09-21,0


In [42]:
grouped = juniper.groupby(['date', 'src_address'])

In [29]:
grouped.ngroups

1251

In [43]:
total = 0
for i in grouped.groups.keys():
    subtotal = np.nansum(juniper.iloc[grouped.groups[i]].sent_size)
    #print (i + ': ' + str(subtotal))
    total += subtotal
    #print i
    #print grouped.groups[i]
print 'total: ' + str(total/Gb) + ' Gb'

total: 24.844180885 Gb


In [86]:
temp_result = []
for d, i in grouped.groups.keys():
    temp_result.append([d, i, len(juniper.iloc[grouped.groups[(d, i)]]), np.sum(juniper.iloc[grouped.groups[(d, i)]]).total_size])

In [93]:
sqldb = pd.DataFrame(temp_result, columns=['date', 'ip', 'count', 'total_size'])

In [96]:
sqldb.head()

,date,ip,count,total_size
0,2015-09-21,141.212.122.204,1,192
1,2015-09-21,122.96.129.236,1,1025
2,2015-09-22,172.20.78.50,6,366
3,2015-09-22,184.105.247.224,1,192
4,2015-09-21,172.20.68.21,55098,8623658517


In [97]:
sqldb.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9, 
            ...
            1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250],
           dtype='int64', length=1251)

In [98]:
sqldb.to_sql('data', disk_engine, if_exists='append', index=False)

OperationalError: (sqlite3.OperationalError) table data has no column named date [SQL: u'INSERT INTO data (date, ip, count, total_size) VALUES (?, ?, ?, ?)'] [parameters: (('2015-09-21', '141.212.122.204', 1L, 192.0), ('2015-09-21', '122.96.129.236', 1L, 1025.0), ('2015-09-22', '172.20.78.50', 6L, 366.0), ('2015-09-22', '184.105.247.224', 1L, 192.0), ('2015-09-21', '172.20.68.21', 55098L, 8623658517.0), ('2015-09-21', '119.56.122.193', 2L, 4147.0), ('2015-09-21', '141.212.122.58', 1L, 20.0), ('2015-09-21', '112.111.3.88', 1L, 1204.0)  ... displaying 10 of 1251 total bound parameter sets ...  ('2015-09-21', '222.76.217.82', 4L, 112.0), ('2015-09-21', '184.105.247.248', 1L, 20.0))]

In [63]:
juniper[grouped.groups[(('2015-09-22 00:00:00'), '172.20.78.252')]]

KeyError: ('2015-09-22 00:00:00', '172.20.78.252')

In [1]:
#for k in grouped.groups.keys():
#    print k

In [26]:
len(juniper.iloc[483167:483169, :])

2

In [27]:
juniper.iloc[483167:483169, :]

,time,device_name,traffic_start,duration,policy_no,service,protocol,src_zone,dst_zone,action,sent_size,received_size,src_address,dst_address,src_port,dst_port,remarks
483167,2015-09-17 11:20:49,hk-SSG140,2015-09-17 11:32:43,0,189,tcp/port:60088,6,Untrust,HK-LAN,Deny,0,20,111.243.205.239,218.188.73.142,12200,60088,session_id=0
483168,2015-09-17 11:20:49,hk-SSG140,2015-09-17 11:32:43,0,189,tcp/port:1095,6,Untrust,HK-LAN,Deny,0,20,111.243.205.239,218.188.73.142,12200,1095,session_id=0


In [28]:
juniper.iloc[925410,]

time             2015-09-17 15:42:17
device_name                hk-SSG140
traffic_start    2015-09-17 15:54:11
duration                           2
policy_no                        519
service                          dns
protocol                          17
src_zone                      HK-LAN
dst_zone                     Untrust
action                        Permit
sent_size                        109
received_size                    197
src_address             172.20.78.20
dst_address                  8.8.8.8
src_port                       56606
dst_port                          53
remarks                   src-xlated
Name: 925410, dtype: object

In [29]:
#pd.DataFrame.sum(juniper[['sent_size']])

In [30]:
#juniper